# Comparison of spatial segmentation algorithms (from scikit-image and orfeo toolbox)

## Introduction

## Data preparation

In [1]:
import os
from pprint import pprint
import requests
import tempfile
import rasterio
import sys

try:
    sys.path.remove('/Library/Python/2.7/site-packages/numpy-override')
except:
    pass

def download(url, outdir):
    filename = os.path.join(outdir, os.path.basename(url).split("?")[0])
    if not os.path.isfile(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as dst:
            for chunk in r.iter_content(chunk_size=1024):
                dst.write(chunk)
    return filename

In [4]:
scene_id = 'LC08_L1TP_029046_20171027_20171108_01_T1'
old_scene_id = 'LC80290462017300LGN00'
url = 'https://earthexplorer.usgs.gov/download/12864/%s/STANDARD/EE' % old_scene_id
url2 = 'https://dds.cr.usgs.gov/ltaauth/hsm/lsat1/collection01/oli_tirs/T1/2017/029/046/LC08_L1TP_029046_20171027_20171108_01_T1.tar.gz?id=3rfs2duqtuo4urjs3vrhvc0gh5&iid=LC80290462017300LGN00&did=357565493&ver=production'


filename = download(url2, '/Users/ldutrieux/sandbox/')
print filename


/Users/ldutrieux/sandbox/LC08_L1TP_029046_20171027_20171108_01_T1.tar.gz


In [3]:
with rasterio.open('/vsitar/%s/%s_B2.TIF' % (filename, scene_id)) as src:
    meta = src.meta
    
pprint(meta)
meta.update(count=6)
stack = '/Users/ldutrieux/sandbox/LC08_L1TP_029046_20171027_20171108_01_T1_stack.tif'

if not os.path.isfile(stack):
    with rasterio.open(stack, 'w', **meta) as dst:
        for id, i in enumerate([2, 3, 4, 5, 6, 7]):
            with rasterio.open('/vsitar/%s/%s_B%d.TIF' % (filename, scene_id, i)) as src:
                band = src.read(1)
                dst.write(band, id + 1)

{'count': 1,
 'crs': CRS({'init': u'epsg:32613'}),
 'driver': u'GTiff',
 'dtype': 'uint16',
 'height': 7721,
 'nodata': None,
 'transform': Affine(30.0, 0.0, 550185.0,
       0.0, -30.0, 2352915.0),
 'width': 7561}


In [20]:
from skimage import segmentation
from rasterio import features
import fiona
import numpy as np

# Read multilayer raster to 3D array. Dimensions order must be x,y,z
with rasterio.open(stack) as src:
    meta = src.meta
    transform = src.transform
    img = np.empty((src.height, src.width, src.count),
                    dtype=src.meta['dtype'])
    for layer_id in range(src.count):
        img[:,:,layer_id] = src.read(layer_id + 1)
        
        
# Image size 


## Slick parameters
compactness = 0.05
n_segments = 100000

segments = segmentation.slic(img, compactness = compactness, n_segments=n_segments, multichannel = True)
# segments = segmentation.felzenszwalb(img, scale = 20, min_size=50)
# segments = segmentation.quickshift(img, convert2lab=False)

# Vectorize
shapes = features.shapes(segments.astype(np.uint16), transform=transform)

# Prepare schema
fiona_kwargs = {'crs': meta['crs'],
                'driver': 'GPKG',
                'schema': {'geometry': 'Polygon',
                'properties': {'id': 'int'}}}

# Write to file
with fiona.open('/Users/ldutrieux/sandbox/segments.gpkg', 'w', layer='slic_%3f_%d' % (compactness, n_segments),
                **fiona_kwargs) as dst:
    for item in shapes:
        feature = {'geometry': item[0], 'properties': {'id': int(item[1])}}
        dst.write(feature)

(7721, 7561, 6)


In [13]:
print fiona.supported_drivers

{'ESRI Shapefile': 'raw', 'OpenFileGDB': 'r', 'SUA': 'r', 'ARCGEN': 'r', 'GeoJSON': 'rw', 'GPKG': 'rw', 'Idrisi': 'r', 'GPX': 'raw', 'SEGY': 'r', 'BNA': 'raw', 'AeronavFAA': 'r', 'GPSTrackMaker': 'raw', 'DGN': 'raw', 'PCIDSK': 'r', 'MapInfo File': 'raw', 'DXF': 'raw'}


In [7]:
import numpy as np
import rasterio
import otbApplication

stack = '/Users/ldutrieux/sandbox/LC08_L1TP_029046_20171027_20171108_01_T1_stack.tif'

with rasterio.open(stack) as src:
    meta = src.meta
    transform = src.transform
    img = np.empty((src.height, src.width, src.count),
                    dtype=src.meta['dtype'])
    for layer_id in range(src.count):
        img[:,:,layer_id] = src.read(layer_id + 1)
        


Seg = otbApplication.Registry.CreateApplication("Segmentation")

# Set segmentation parameters
Seg.SetVectorImageFromNumpyArray('in', img)

Seg.SetParameterString("mode","vector")

Seg.SetParameterString("mode.vector.out", '/Users/ldutrieux/sandbox/test.sqlite')
Seg.SetParameterString("mode.vector.layername", 'meanshift')
Seg.SetParameterString("mode.vector.fieldname", "id")
Seg.SetParameterInt("mode.vector.startlabel", 1)
Seg.SetParameterString("mode.vector.outmode", "ulu")
Seg.SetParameterStringList("mode.vector.ogroptions", ["SPATIALITE=yes"])
Seg.SetParameterInt("mode.vector.tilesize", 3000)

Seg.SetParameterString("filter","meanshift")
Seg.SetParameterInt("filter.meanshift.spatialr", 3) 
Seg.SetParameterFloat("filter.meanshift.ranger", 2)
Seg.SetParameterInt("filter.meanshift.minsize", 20)

# The following line execute the application
Seg.ExecuteAndWriteOutput()

0